# Архитектура GPT-2

GPT-2 — это эволюция GPT-1, предложенная OpenAI в 2019 году. Модель сохраняет **архитектуру трансформера-декодера**, но вносит несколько ключевых улучшений, благодаря которым она стала более стабильной и способной генерировать длинные тексты.

---

## Основные улучшения GPT-2 по сравнению с GPT-1

### 1. Масштаб модели

- GPT-2 значительно **увеличила количество параметров**.
    
    |Модель|Параметры|Слои (Decoder)|Размер эмбеддингов|Heads|
    |---|---|---|---|---|
    |GPT-1|117M|12|768|12|
    |GPT-2|1.5B|48|1600|25|
    
- Увеличение глубины и ширины слоёв позволяет модели **захватывать более сложные закономерности языка**.
    

---

### 2. Pre-norm и Post-norm

![](https://ucarecdn.com/b7f2a1e5-620d-4efc-989f-2348a613ffb4/)

- **GPT-1** использовала **post-norm**, когда слои нормализации применялись **после блоков внимания и FFN**.
    
- **GPT-2** ввела **pre-norm**, то есть **слои нормализации располагаются перед блоками внимания и FFN**.
    
    - Это повышает **устойчивость обучения глубоких сетей**, особенно при увеличении числа слоёв.
        
    - Также добавлен **один слой нормализации после последнего блока декодера**, что стабилизирует выход модели.
        

---

### 3. GELU вместо ReLU

![](https://ucarecdn.com/c8bbc3fb-6951-4f2b-aed9-944e0612ab3c/)

- В GPT-1 использовалась **ReLU** в полносвязных сетях (FFN).
    
- В GPT-2 применяют **GELU (Gaussian Error Linear Unit)**:


![](https://ucarecdn.com/d9469f32-11eb-46ad-a6fb-e6f4735e847a/)
    
$$ 
\text{GELU}(x) = x \cdot \Phi(x)  
$$

где $Phi(x)$ — функция нормального распределения.

- GELU **плавно подавляет отрицательные значения**, создавая мягкий переход около нуля.
    
- Эмпирически улучшает **скорость обучения и качество генерации** текста.
    

---

### 4. KV-cache (Key-Value Cache)

- GPT-2 использует **оптимизацию вычислений при генерации текста**:
    
    - Ранее в GPT-1 каждый прогон модели пересчитывал **всё внимание** заново для всей последовательности.
        
    - KV-cache позволяет **сохранять Q, K, V для уже обработанных токенов** и обновлять только новые токены.
        
    - Это значительно ускоряет **генерацию длинных текстов**.
        

---

### 5. Tokenization и словарь

- GPT-2 сохраняет **Byte Pair Encoding (BPE)**, но словарь **больше (50 000 токенов)**.
    
- Это позволяет модели **обрабатывать редкие слова, спецсимволы и эмодзи**.
    

---

### 6. Маскированное внимание (Causal Self-Attention)

- GPT-2 продолжает использовать **авторегрессионное предсказание**: каждый токен зависит только от предыдущих.
    
- Отличие в **оптимизации для больших последовательностей** и увеличении числа голов внимания, что повышает способность захватывать сложные зависимости между токенами.
    

---

### 7. Feed-Forward Network (FFN)

- Двухслойная FFN с **GELU** и шириной 4× размер эмбеддингов.
    
- Позволяет **обрабатывать и смешивать информацию из разных голов внимания** более эффективно, чем ReLU в GPT-1.
    

---

### 8. Генерация текста

- GPT-2 остаётся **авторегрессионной**, как GPT-1.
    
- Улучшения:
    
    - KV-cache для ускорения генерации длинных последовательностей.
        
    - Поддержка **top-k и top-p (nucleus) sampling** для управления разнообразием текста.
        
    - Более длинные контексты (до 1024 токенов и более).
        

---

### 🔹 Сравнение GPT-1 и GPT-2

|Компонент|GPT-1|GPT-2|
|---|---|---|
|Слои Decoder|12|48|
|Эмбеддинги|768|1600|
|Heads|12|25|
|Словарь|~40k|50k|
|Max Seq Len|512|1024|
|LayerNorm|Post-LN|Pre-LN + финальный LN|
|Активация FFN|ReLU|GELU|
|Генерация|Полный расчет заново|KV-cache + top-k/top-p|


In [2]:
import dill
from torch import nn
import torch

## BPE Tokenizator

In [3]:
class BPE:
    def __init__(self, vocab_size: int):
        self.vocab_size = vocab_size
        self.id2token = {}
        self.token2id = {}

    def fit(self, text: str):
        # 1. Получаем уникальные токены (символы)
        unique_tokens = sorted(set(text))
        tokens = unique_tokens.copy()

        # 2. Разбиваем текст на токены-символы
        sequence = list(text)

        # 3. Объединяем токены до достижения нужного размера словаря
        while len(tokens) < self.vocab_size:
            #print(f'len={len(tokens)} < {self.vocab_size}')
            # Считаем частоты пар
            pair_freq = {}
            for i in range(len(sequence) - 1):
                pair = (sequence[i], sequence[i + 1])
                #print(f'pair = {pair}')
                if pair not in pair_freq:
                    pair_freq[pair] = 0
                pair_freq[pair] += 1


            #print(f'pair_freq = {pair_freq}')  
            if not pair_freq:
                break  # нет пар — выходим

            #for x in pair_freq.items():
            #    self.debug(x, sequence)

            # Находим самую частую пару (в случае равенства — та, что встретилась первой)
            most_frequent_pair = max(pair_freq.items(), key=lambda x: (x[1], -self._pair_first_index(sequence, x[0])))[0]
            #print(most_frequent_pair)
            # Создаем новый токен
            new_token = most_frequent_pair[0] + most_frequent_pair[1]
            #print(f"new token={new_token}")
            tokens.append(new_token)
            #print(f"tokens={tokens}")

            i = 0
            new_sequence = []

            while i < len(sequence):
                if i < len(sequence) - 1 and (sequence[i], sequence[i + 1]) == most_frequent_pair:
                    new_sequence.append(new_token)
                    i += 2  # пропускаем два символа — заменённую пару
                else:
                    new_sequence.append(sequence[i])
                    i += 1
            sequence = new_sequence
            #break
        
        # 4. Создаем словари
        self.vocab = tokens.copy()
        self.token2id = dict(zip(tokens, range(self.vocab_size)))
        self.id2token = dict(zip(range(self.vocab_size), tokens))

    def _pair_first_index(self, sequence, pair):
        for i in range(len(sequence) - 1):
            if (sequence[i], sequence[i + 1]) == pair:
                return i
        return float('inf')  # если пара не найдена (в теории не должно случиться)


    def encode(self, text: str):
        # 1. Разбиваем текст на токены-символы
        sequence = list(text)
        # 2. Инициализация пустого списка токенов
        tokens = []
        # 3. Установить i = 0
        i = 0
        while i < len(text):
            # 3.1 Найти все токены в словаре, начинающиеся с text[i]
            start_char = text[i]
            result = [token for token in self.vocab if token.startswith(start_char)]
            # 3.2 Выбрать самый длинный подходящий токен
            find_token = self._find_max_matching_token(text[i:], result)
            if find_token is None:
                # Обработка неизвестного символа
                tokens.append(text[i])  # Добавляем сам символ как токен
                i += 1
            else:
                # 3.3 Добавить токен в результат
                tokens.append(find_token)
                # 3.4 Увеличить i на длину токена
                i += len(find_token)

        # 4. Заменить токены на их ID
        return self._tokens_to_ids(tokens)

    def _find_max_matching_token(self, text: str, tokens: list):
        """Находит самый длинный токен из списка, с которого начинается текст"""
        matching = [token for token in tokens if text.startswith(token)]
        return max(matching, key=len) if matching else None

    def _tokens_to_ids(self, tokens):
        """Конвертирует список токенов в их ID с обработкой неизвестных токенов"""
        ids = []
        for token in tokens:
            if token in self.token2id:
                ids.append(self.token2id[token])
            else:
                ids.append(0)  # Специальное значение
        return ids


    def decode(self, ids: list) -> str:
        return ''.join(self._ids_to_tokens(ids))

    def _ids_to_tokens(self, ids: list) -> list:
        """Конвертирует список Ids в их tokens"""
        tokens = []
        for id in ids:
            if id in self.id2token:
                tokens.append(self.id2token[id])
            else:
                tokens.append('')  # Специальное значение
        return tokens


    def save(self, filename):
        with open(filename, 'wb') as f:
            dill.dump(self, f)
        print(f"Объект сохранён в {filename}")


    @classmethod
    def load(cls, filename):
        with open(filename, 'rb') as f:
            obj = dill.load(f)
                
        print(f"Объект загружен из {filename}")
        return obj

## GPT2

In [9]:
import torch
from torch import nn
import torch.nn.functional as F
from math import sqrt
import torch
from torch import nn
from torch import Tensor

class TokenEmbeddings(nn.Module):
    def __init__(self, vocab_size: int, emb_size: int):
        super().__init__()
        self._embedding = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=emb_size
        )

    def forward(self, x: Tensor) -> Tensor:
        return self._embedding(x)

    @property
    def num_embeddings(self) -> int:
        return self._embedding.num_embeddings

    @property
    def embedding_dim(self) -> int:
        return self._embedding.embedding_dim


import torch
from torch import nn, Tensor

class PositionalEmbeddings(nn.Module):
    def __init__(self, max_seq_len: int, emb_size: int):
        super().__init__()
        self.max_seq_len = max_seq_len
        self.emb_size = emb_size
        self.embedding = nn.Embedding(
            num_embeddings=max_seq_len,
            embedding_dim=emb_size
        )

    def forward(self, seq_len: int, start_pos: int = 0) -> Tensor:
        if seq_len < 1 or seq_len > self.max_seq_len:
            raise IndexError(f"Длина {seq_len} должна быть от 1 до {self.max_seq_len}")
        if start_pos == 0:
            positions = torch.arange(seq_len, device=self.embedding.weight.device)
        else:
            positions = torch.arange(start=start_pos, end=start_pos + seq_len, device=self.embedding.weight.device)
        return self.embedding(positions)
    
    
class HeadAttention(nn.Module):

    def __init__(self, emb_size: int, head_size: int, max_seq_len: int):
        super().__init__()
        self._emb_size = emb_size
        self._head_size = head_size
        self._max_seq_len = max_seq_len

        self._k = nn.Linear(emb_size, head_size)
        self._q = nn.Linear(emb_size, head_size)
        self._v = nn.Linear(emb_size, head_size)

        mask = torch.tril(torch.ones(max_seq_len, max_seq_len))
        self.register_buffer('_tril_mask', mask.bool() if hasattr(torch, 'bool') else mask.byte())

    def forward(self, x: torch.Tensor, use_cache: bool = True, cache: tuple = None) -> tuple:
        seq_len = x.shape[1]
        if seq_len > self._max_seq_len:
            raise ValueError(f"Длина последовательности {seq_len} превышает максимум {self._max_seq_len}")

        k = self._k(x)  # [B, T, hs]
        q = self._q(x)  # [B, T, hs]
        v = self._v(x)  # [B, T, hs]

        if cache is not None:
            k_cache, v_cache = cache
            k = torch.cat([k_cache, k], dim=1)  # [B, cache_len + T, hs]
            v = torch.cat([v_cache, v], dim=1)  # [B, cache_len + T, hs]
        
        scores = q @ k.transpose(-2, -1) / sqrt(self._head_size)
        
        if cache is None:
            scores = scores.masked_fill(~self._tril_mask[:seq_len, :seq_len], float('-inf'))
        
        weights = F.softmax(scores, dim=-1)
        x_out = weights @ v  # [B, T, hs]

        if use_cache is True:
            return (x_out, (k, v))
        else:
            return (x_out, None)
        
from torch import nn
import torch
import math

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads: int, emb_size: int, head_size: int, max_seq_len: int, dropout: float = 0.1):

        super().__init__()
        self._heads = nn.ModuleList([
            HeadAttention(
                emb_size=emb_size, 
                head_size=head_size, 
                max_seq_len=max_seq_len
            ) for _ in range(num_heads)
        ])
        self._layer = nn.Linear(head_size * num_heads, emb_size)
        self._dropout = nn.Dropout(dropout)

    def forward(self, x: torch.Tensor, mask: torch.Tensor = None, use_cache: bool = True, cache: list = None):

        attention_results = []
        for i, head in enumerate(self._heads):
            head_cache = cache[i] if cache is not None else None
            result = head(x, use_cache=use_cache, cache=head_cache)
            attention_results.append(result)
        
        outputs, caches = zip(*attention_results)
        attention_outputs = list(outputs)
        kv_caches = list(caches)
 
        concatenated_attention = torch.cat(attention_outputs, dim=-1)

        projected_output = self._layer(concatenated_attention)
        
        final_output = self._dropout(projected_output)
        
        if use_cache is True:
            return (final_output, kv_caches)
        else:
            return (final_output, None)


class GELU(nn.Module):
    def __init__(self):
        super().__init__()
        self.sqrt_2_over_pi = torch.sqrt(torch.tensor(2.0) / math.pi)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return 0.5 * x * (1 + torch.tanh(
            self.sqrt_2_over_pi * (x + 0.044715 * torch.pow(x, 3))
        ))

class FeedForward(nn.Module):

    def __init__(self, emb_size: int, dropout: float = 0.1):
        super().__init__()
        self._layer1 = nn.Linear(emb_size, emb_size * 4)
        self._gelu = GELU()
        self._layer2 = nn.Linear(emb_size * 4, emb_size)
        self._dropout = nn.Dropout(dropout)

    def forward(self, x: torch.Tensor):
        input_dtype = x.dtype
        
        if input_dtype != self._layer1.weight.dtype:
            self._layer1 = self._layer1.to(dtype=input_dtype)
            self._layer2 = self._layer2.to(dtype=input_dtype)
            
        x = self._layer1(x)
        x = self._gelu(x)
        x = self._layer2(x)
        return self._dropout(x)
    
class Decoder(nn.Module):
    def __init__(self, 
        num_heads: int,
        emb_size: int,
        head_size: int,
        max_seq_len: int,
        dropout: float = 0.1
    ):
        super().__init__()
        self._heads = MultiHeadAttention(
            num_heads=num_heads, 
            emb_size=emb_size, 
            head_size=head_size, 
            max_seq_len=max_seq_len, 
            dropout=dropout
        )
        self._ff = FeedForward(emb_size=emb_size, dropout=dropout)
        self._norm1 = nn.LayerNorm(emb_size)
        self._norm2 = nn.LayerNorm(emb_size)

    def forward(self, x: torch.Tensor, mask: torch.Tensor = None, use_cache: bool = True, cache: list = None) -> torch.Tensor:
        norm1_out = self._norm1(x)
        attention, kv_caches = self._heads(norm1_out, mask, use_cache=use_cache, cache=cache)
        out = attention + x
        
        norm2_out = self._norm2(out)
        ffn_out = self._ff(norm2_out)

        if use_cache is True:
            return (ffn_out + out, kv_caches)
        else:
            return (ffn_out + out, None)



from torch import nn
import torch
import torch.nn.functional as F

class GPT2(nn.Module):
    def __init__(self,
        vocab_size: int,
        max_seq_len: int,
        emb_size: int,
        num_heads: int,
        head_size: int,
        num_layers: int,
        dropout: float = 0.1,
        device: str = 'cpu'
    ):
        super().__init__()
        self._vocab_size = vocab_size
        self._max_seq_len = max_seq_len
        self._emb_size = emb_size
        self._num_heads = num_heads
        self._head_size = head_size
        self._num_layers = num_layers
        self._dropout = dropout
        self._device = device
        
        self.validation_loss = None

        # Инициализация слоев
        self._token_embeddings = TokenEmbeddings(
            vocab_size=vocab_size, 
            emb_size=emb_size
        )
        self._position_embeddings = PositionalEmbeddings(
            max_seq_len=max_seq_len, 
            emb_size=emb_size
        )
        self._dropout = nn.Dropout(dropout)
        self._decoders = nn.ModuleList([Decoder(
            num_heads=num_heads,
            emb_size=emb_size,
            head_size=head_size,
            max_seq_len=max_seq_len,
            dropout=dropout 
        ) for _ in range(num_layers)])
        self._norm = nn.LayerNorm(emb_size)
        self._linear = nn.Linear(emb_size, vocab_size)

    def forward(self, x: torch.Tensor, use_cache: bool = True, cache: list = None) -> tuple:
        # Проверка длины последовательности (только при отсутствии кэша)
        if cache is None and x.size(1) > self._max_seq_len:
            raise ValueError(f"Длина последовательности {x.size(1)} превышает максимальную {self.max_seq_len}")
        
        
        # Вычисление start_pos из кэша (если кэш передан)
        if cache is not None:
            # При кэше обрабатываем только один токен (последний)
            seq_len = 1
            # Вычисляем start_pos из самого нижнего уровня кэша
            if cache and cache[0] and cache[0][0]:
                key_cache, _ = cache[0][0]  # Первый декодер, первая голова
                start_pos = key_cache.size(1)  # cache_len
            else:
                start_pos = 0
        else:
            # Без кэша работаем как раньше
            start_pos = 0
            seq_len = x.size(1)

        # Эмбеддинги токенов и позиций
        tok_out = self._token_embeddings(x)  # [batch, seq_len, emb_size]
        pos_out = self._position_embeddings(seq_len, start_pos=start_pos)  # [seq_len, emb_size]
        
        # Комбинирование
        out = self._dropout(tok_out + pos_out.unsqueeze(0))  # [batch, seq_len, emb_size]
        
        # Стек декодеров с передачей кэша
        new_cache = []
        for i, decoder in enumerate(self._decoders):
            decoder_cache = cache[i] if cache is not None else None
            decoder_result = decoder(out, use_cache=use_cache, cache=decoder_cache)

            # Извлекаем результат из кортежа
            if use_cache:
                out, decoder_new_cache = decoder_result
                new_cache.append(decoder_new_cache)
            else:
                out = decoder_result[0]

        out = self._norm(out)
        logits = self._linear(out)
            
        # Возвращаем результат с учетом use_cache
        if use_cache:
            return (logits, new_cache)
        else:
            return (logits, None)

    def generate(self,
        x: torch.Tensor, 
        max_new_tokens: int, 
        do_sample: bool,
        temperature: float = 1.0,
        top_k: int = None,
        top_p: float = None,
        use_cache: bool = True
    ) -> torch.Tensor:
        cache = None

        for _ in range(max_new_tokens):
            if use_cache and cache is not None:
                # Используем кэш - передаем только последний токен
                x_input = x[:, -1:]  # [batch_size, 1]
            else:
                # Первая итерация или кэш отключен - передаем всю последовательность
                x_input = x
            
            # Прямой проход с кэшем
            logits, new_cache = self.forward(x_input, use_cache=use_cache, cache=cache)
            
            # Обновляем кэш для следующей итерации
            if use_cache:
                cache = new_cache

            last_logits = logits[:, -1, :]  # [batch_size, vocab_size]

            # Масштабируем логиты температурой
            if temperature > 0:
                logits_scaled = last_logits / temperature
            else:
                logits_scaled = last_logits

            if do_sample == True and top_k != None:
                _, topk_indices = torch.topk(logits_scaled, top_k, dim=-1)

                # # Заменим все НЕ top-k логиты на -inf
                masked_logits = logits_scaled.clone()
                vocab_size = logits_scaled.size(-1)

                # создаём маску: 1, если токен НЕ в topk_indices
                mask = torch.ones_like(logits_scaled, dtype=torch.uint8)
                mask.scatter_(1, topk_indices, 0)  # 0 там, где top-k индексы
                masked_logits[mask.byte()] = float('-inf')

                logits_scaled = masked_logits

            if do_sample == True and top_p != None:
                # 1. Применим softmax, чтобы получить вероятности:
                probs = F.softmax(logits_scaled, dim=-1)  # [B, vocab_size]
                # 2. Отсортируем токены по убыванию вероятностей:
                sorted_probs, sorted_indices = torch.sort(probs, descending=True, dim=-1)
                # 3. Посчитаем кумулятивную сумму вероятностей:
                cum_probs = torch.cumsum(sorted_probs, dim=-1)  # [B, vocab_size]
                # 4. Определим маску: оставить токены, пока сумма < top_p
                sorted_mask = (cum_probs <= top_p).byte()  # [B, vocab_size]
                # Гарантируем, что хотя бы первый токен останется
                sorted_mask[:, 0] = 1
                # 5. Преобразуем маску обратно в оригинальный порядок:
                # Создаём полную маску из 0
                mask = torch.zeros_like(probs, dtype=torch.uint8)
                # Устанавливаем 1 в местах нужных токенов
                mask.scatter_(dim=1, index=sorted_indices, src=sorted_mask)
                # 6. Зануляем логиты токенов вне топ-p:
                logits_scaled[~mask] = float('-inf')

            # 4. Применяем Softmax
            probs = F.softmax(logits_scaled, dim=-1)  # [batch_size, vocab_size]


            if do_sample == True:
                # 5. Если do_sample равен True, то отбираем токен случайно с помощью torch.multinomial
                next_token = torch.multinomial(probs, num_samples=1)  # [batch_size, 1]
            else:
                # 5. Если do_sample равен False, то выбираем токен с максимальной вероятностью
                next_token = torch.argmax(probs, dim=-1, keepdim=True)  # [batch_size, 1]
            
            # 6. Добавляем его к последовательности
            x = torch.cat([x, next_token], dim=1)  # [batch_size, seq_len+1]
        return x

    def save(self, path):
        torch.save({
            'model_state_dict': self.state_dict(),
            'vocab_size': self._vocab_size,
            'max_seq_len': self._max_seq_len,
            'emb_size': self._emb_size,
            'num_heads': self._num_heads,
            'head_size': self._head_size,
            'num_layers': self._num_layers
        }, path)

    @classmethod
    def load(cls, path, device):
        checkpoint = torch.load(path, map_location=device)
        model = cls(
            vocab_size=checkpoint['vocab_size'],
            max_seq_len=checkpoint['max_seq_len'],
            emb_size=checkpoint['emb_size'],
            num_heads=checkpoint['num_heads'],
            head_size=checkpoint['head_size'],
            num_layers=checkpoint['num_layers']
        )
        model.load_state_dict(checkpoint['model_state_dict'])
        model.to(device)
        return model

    @property
    def max_seq_len(self) -> int:
        return self._max_seq_len

## 2. Обучение GPT-2

GPT-2 обучается в два этапа:

- 1️⃣ **Предобучение (Unsupervised Pretraining)**  
- 2️⃣ **Дообучение (Supervised Fine-Tuning)**




### 5.1 Предобучение

На первом этапе модель обучается без разметки: она получает большой корпус текстов и учится **предсказывать следующий токен** по предыдущим.

Функция потерь:
$$
L = - \sum_{t=1}^{T} \log P(x_t | x_1, x_2, ..., x_{t-1})
$$

Таким образом, модель учится строить вероятностную модель языка, "угадывая" продолжение текста.


Во время **предобучения** GPT-1 учится **предсказывать следующий токен** (language modeling task).  
Формально:  
$$ 
P(x_t ,|, x_1, x_2, \dots, x_{t-1})  
$$ 
То есть, если на вход подаётся предложение `"I love deep"`, модель должна предсказать `"learning"`.


### ✅ 5.1.1 Подготовка данных

Создадим **датасет** на основе BPE-токенизатора:

In [10]:
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDataset(Dataset):
    def __init__(self, text: str, bpe: BPE, block_size: int):
        self.bpe = bpe
        self.block_size = block_size
        self.data = bpe.encode(text)
        
    def __len__(self):
        return len(self.data) - self.block_size

    def __getitem__(self, idx):
        x = torch.tensor(self.data[idx:idx+self.block_size], dtype=torch.long)
        y = torch.tensor(self.data[idx+1:idx+self.block_size+1], dtype=torch.long)
        return x, y

- `x` — входная последовательность токенов
    
- `y` — та же последовательность, но сдвинутая на один токен вперёд (цель)

### ✅ 5.1.2 Цикл обучения

Для обучения создадим функцию:

In [15]:
import torch.nn.functional as F
from torch import optim

def train_gpt(model, dataset, epochs=5, batch_size=32, lr=3e-4, device='cpu'):
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    optimizer = optim.AdamW(model.parameters(), lr=lr)

    model.to(device)
    model.train()

    for epoch in range(epochs):
        total_loss = 0
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)

            # Прямой проход
            logits, _ = model(x, use_cache=False)  # [B, T, vocab_size]

            # Перестроим выход под CrossEntropy
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), y.view(-1))

            # Обратное распространение
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(dataloader)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")

    return model

### ✅ 5.1.3 Пример запуска


**🧠 Конфигурация GPT-2 Mini (официальная OpenAI)**


| Параметр        | Значение | Описание                                      |
| --------------- | -------- | --------------------------------------------- |
| **vocab_size**  | `50257`  | Размер словаря (BPE токенизатор OpenAI)       |
| **max_seq_len** | `512`   | Максимальная длина входной последовательности |
| **emb_size**    | `256`    | Размер эмбеддингов (векторное пространство)   |
| **num_heads**   | `4`     | Количество голов в multi-head attention       |
| **head_size**   | `64`     | Размерность одной головы внимания (768 / 12)  |
| **num_layers**  | `4`     | Количество блоков (декодеров)                 |
| **dropout**     | `0.1`    | Вероятность дропаута                          |


In [30]:
# 1. Исходный текст
text = "Deep learning is amazing. Transformers changed the world. Attention is all you need. GPT models revolutionized NLP."

# 2. Обучаем токенизатор
bpe = BPE(vocab_size=100)
bpe.fit(text)

# 3. Создаем датасет
dataset = GPTDataset(text, bpe, block_size=8)
print(f"Dataset length: {len(dataset)}")

# 4. Инициализируем модель
gpt = GPT2(
    vocab_size=len(bpe.vocab),  # размер словаря BPE
    max_seq_len=512,           # GPT-2 использует контекст в 512 токена
    emb_size=256,               # размер эмбеддингов
    num_heads=4,               # количество голов внимания
    head_size=64,               # размер каждой головы (256 / 4)
    num_layers=4,              # количество блоков Transformer
    dropout=0.1                 # стандартный dropout GPT-2
)

# 5. Обучаем
train_gpt(gpt, dataset, epochs=100, batch_size=4)

Dataset length: 20
Epoch 1/100, Loss: 4.0049
Epoch 2/100, Loss: 2.2952
Epoch 3/100, Loss: 1.2738
Epoch 4/100, Loss: 0.6864
Epoch 5/100, Loss: 0.4070
Epoch 6/100, Loss: 0.3075
Epoch 7/100, Loss: 0.2422
Epoch 8/100, Loss: 0.1881
Epoch 9/100, Loss: 0.1484
Epoch 10/100, Loss: 0.1258
Epoch 11/100, Loss: 0.1153
Epoch 12/100, Loss: 0.1039
Epoch 13/100, Loss: 0.0852
Epoch 14/100, Loss: 0.0897
Epoch 15/100, Loss: 0.0799
Epoch 16/100, Loss: 0.0741
Epoch 17/100, Loss: 0.0809
Epoch 18/100, Loss: 0.0680
Epoch 19/100, Loss: 0.0717
Epoch 20/100, Loss: 0.0648
Epoch 21/100, Loss: 0.0684
Epoch 22/100, Loss: 0.0654
Epoch 23/100, Loss: 0.0631
Epoch 24/100, Loss: 0.0686
Epoch 25/100, Loss: 0.0633
Epoch 26/100, Loss: 0.0624
Epoch 27/100, Loss: 0.0618
Epoch 28/100, Loss: 0.0686
Epoch 29/100, Loss: 0.0613
Epoch 30/100, Loss: 0.0564
Epoch 31/100, Loss: 0.0587
Epoch 32/100, Loss: 0.0696
Epoch 33/100, Loss: 0.0574
Epoch 34/100, Loss: 0.0594
Epoch 35/100, Loss: 0.0556
Epoch 36/100, Loss: 0.0630
Epoch 37/100, Loss

GPT2(
  (_token_embeddings): TokenEmbeddings(
    (_embedding): Embedding(100, 256)
  )
  (_position_embeddings): PositionalEmbeddings(
    (embedding): Embedding(512, 256)
  )
  (_dropout): Dropout(p=0.1, inplace=False)
  (_decoders): ModuleList(
    (0-3): 4 x Decoder(
      (_heads): MultiHeadAttention(
        (_heads): ModuleList(
          (0-3): 4 x HeadAttention(
            (_k): Linear(in_features=256, out_features=64, bias=True)
            (_q): Linear(in_features=256, out_features=64, bias=True)
            (_v): Linear(in_features=256, out_features=64, bias=True)
          )
        )
        (_layer): Linear(in_features=256, out_features=256, bias=True)
        (_dropout): Dropout(p=0.1, inplace=False)
      )
      (_ff): FeedForward(
        (_layer1): Linear(in_features=256, out_features=1024, bias=True)
        (_gelu): GELU()
        (_layer2): Linear(in_features=1024, out_features=256, bias=True)
        (_dropout): Dropout(p=0.1, inplace=False)
      )
      (_nor


---

### 5.2 Дообучение

После предобучения GPT-1 уже знает структуру и грамматику языка.  
На втором этапе она дообучается на конкретных задачах (например, классификация, QA) с помощью размеченных данных.

Технически это почти то же обучение, только:

- Загружаем модель с уже обученными весами.
- Используем новые данные.
- Можно уменьшить скорость обучения.
- Иногда замораживают часть слоёв (например, эмбеддинги).


In [31]:
def fine_tune_gpt(model, dataset, epochs=3, batch_size=16, lr=1e-5, device='cpu', freeze_embeddings=True):
    if freeze_embeddings:
        for param in model._token_embeddings.parameters():
            param.requires_grad = False
        for param in model._position_embeddings.parameters():
            param.requires_grad = False

    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)

    model.to(device)
    model.train()

    for epoch in range(epochs):
        total_loss = 0
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            logits, _ = model(x, use_cache=False)
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), y.view(-1))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Fine-tune Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(dataloader):.4f}")

In [32]:
# Например, мы хотим дообучить модель на стиле коротких технических фраз
fine_tune_text = """
Transformers revolutionize NLP.
Deep learning enables self-attention.
GPT generates text autoregressively.
"""

dataset = GPTDataset(fine_tune_text, bpe, block_size=8)


# Запуск дообучения
fine_tune_gpt(gpt, dataset, epochs=10, batch_size=4, lr=1e-4)

Fine-tune Epoch 1/10, Loss: 4.6839
Fine-tune Epoch 2/10, Loss: 2.7124
Fine-tune Epoch 3/10, Loss: 2.0318
Fine-tune Epoch 4/10, Loss: 1.6738
Fine-tune Epoch 5/10, Loss: 1.4043
Fine-tune Epoch 6/10, Loss: 1.1781
Fine-tune Epoch 7/10, Loss: 1.0102
Fine-tune Epoch 8/10, Loss: 0.8826
Fine-tune Epoch 9/10, Loss: 0.7884
Fine-tune Epoch 10/10, Loss: 0.7057


## 📝 6. Генерация текста после обучения

In [33]:
def generate_text(model, bpe, prompt: str, max_new_tokens=20, device='cpu'):
    model.eval()
    ids = torch.tensor([bpe.encode(prompt)], dtype=torch.long).to(device)
    out = model.generate(ids, max_new_tokens=max_new_tokens, do_sample=True)
    text = bpe.decode(out[0].tolist())
    return text

In [34]:
print(generate_text(gpt, bpe, "Deep learning", max_new_tokens=20))

Deep learning enaten. tns st GP. N
